In [ ]:
!pip install --quiet --upgrade langchain
!pip install --quiet pypdf
!pip install --quiet load_dotenv
!pip install --quiet cohere tiktoken openai
! pip install --quiet faiss-cpu
! pip install --quiet unstructured
! pip install --quiet pdf2image
! pip install --quiet PyPDF2
! pip install chromadb
! pip install pdfminer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.4/277.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 82.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# check if the path exists
import dotenv
import os
os.path.isdir('/content/drive/MyDrive/Colab/Transformer')

True

In [ ]:

dotenv.load_dotenv(os.path.join('/content/drive/MyDrive/Colab/Transformer', '.env'))

True

In [ ]:
import langchain
import openai
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.cache import CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import OnlinePDFLoader
import tiktoken
import requests
from langchain.storage import LocalFileStore
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.cache import CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import TokenTextSplitter
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
import glob
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from requests.exceptions import ReadTimeout, ConnectionError, ConnectTimeout
from PyPDF2 import PdfReader
from langchain.docstore.document import Document
from langchain.document_loaders.pdf import PDFPlumberLoader, PyPDFLoader
from langchain.document_loaders import TextLoader
from langchain.vectorstores import Chroma

# make documents one single PDF first and then load to textloader + splitdocumetns

In [ ]:
def find_latest_txt_file(directory):
    # Get a list of text files in the specified directory
    txt_files = glob.glob(os.path.join(directory, '*.txt'))

    if not txt_files:
        return None  # No text files found in the directory

    # Sort the text files by modification time (newest first)
    txt_files.sort(key=os.path.getmtime, reverse=True)

    # Return the path of the latest text file
    return txt_files[0]


# where our embeddings will be stored
store = LocalFileStore("./cache/")

# Path to the text file containing PDF links
dirname = '/content/drive/MyDrive/Colab/Transformer'
LINK_DIRECTORY = os.path.join(dirname,'link_collection')
text_file_path = find_latest_txt_file(LINK_DIRECTORY)

# Read the text file and load the PDFs
with open(text_file_path, 'r') as file:
    pdf_links = file.read().splitlines()

In [ ]:
# store all the pdfs under data folder

import os
import requests

# Create a "data" folder if it doesn't exist
if not os.path.exists("data"):
    os.makedirs("data")

for pdf_link in pdf_links:
    try:
        response = requests.get(pdf_link, timeout=10)  # Adjust the timeout value as needed
        if response.status_code == 200:
            # Extract the filename from the URL
            filename = pdf_link.split("/")[-1]
            # Combine with "data/" to specify the folder
            file_path = os.path.join("data", filename)

            with open(file_path, 'wb') as f:
                f.write(response.content)
                print(f"PDF link {pdf_link} downloaded and saved as {file_path}")
        else:
            print(f"PDF link {pdf_link} returned status code {response.status_code}. Skipping...")
    except Exception as e:
        print(f"Error processing {pdf_link}: {e}")

PDF link https://www.researchgate.net/profile/Real-Carbonneau/publication/220208887_Machine_Learning-Based_Demand_Forecasting_in_Supply_Chains/links/57c4463a08ae32a03dad3de9/Machine-Learning-Based-Demand-Forecasting-in-Supply-Chains.pdf returned status code 403. Skipping...
PDF link https://www.academia.edu/download/50837650/Application_of_machine_learning_techniqu20161211-20061-1tgykkg.pdf returned status code 404. Skipping...
Error processing https://www.hindawi.com/journals/complexity/2019/9067367/abs/: [Errno 21] Is a directory: 'data/'
PDF link https://journal.oscm-forum.org/journal/journal/download/20201206171323_Paper_1_Vol._14_No_._1,_2021_.pdf downloaded and saved as data/20201206171323_Paper_1_Vol._14_No_._1,_2021_.pdf
PDF link https://www.sciencedirect.com/science/article/pii/S1877050921001654/pdf?md5=c7ce42d7736c04e5fd2f97f3ba2acbdf&pid=1-s2.0-S1877050921001654-main.pdf returned status code 403. Skipping...
PDF link https://www.academia.edu/download/92962396/00207543.2020.1

KeyboardInterrupt: ignored

In [ ]:
from PyPDF2 import PdfMerger
import requests
import sys
import os

# Create a "data" folder if it doesn't exist
if not os.path.exists("data"):
    os.makedirs("data")

merger = PdfMerger()
for pdf_link in pdf_links:
    try:
        response = requests.get(pdf_link, timeout=10)  # Adjust the timeout value as needed
        if response.status_code == 200:
            # Extract the filename from the URL
            filename = pdf_link.split("/")[-1]
            # Combine with "data/" to specify the folder
            file_path = os.path.join("data", filename)
            with open(file_path, 'wb') as f:
                f.write(response.content)
                print(f"PDF link {pdf_link} downloaded and saved as {file_path}")

        else:
            print(f"PDF link {pdf_link} returned status code {response.status_code}. Skipping...")
    # except PdfReadError as e:
    #         print(title)
    #         sys.exit()

    except Exception as e:
        print(f"Error processing {pdf_link}: {e}")


merger.append(filename)
# Save the merged PDF as "result.pdf"
merger.write("result.pdf")
merger.close()


In [ ]:
import os
from PyPDF2 import PdfMerger

# List all PDF files in the "data" folder
pdfs = [os.path.join("data", filename) for filename in os.listdir("data") if filename.endswith(".pdf")]

merger = PdfMerger()

for pdf in pdfs:
    try:
        merger.append(pdf)
    except Exception as e:
        print(f"Error appending {pdf}: {e}")

# Save the merged PDF as "result.pdf"
merger.write("result.pdf")
merger.close()


In [ ]:
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import resolve1

# check the length of pages


file = open("/content/result.pdf", 'rb')
parser = PDFParser(file)
document = PDFDocument(parser)

# This will give you the count of pages
print(resolve1(document.catalog['Pages'])['Count'])

In [ ]:
# Create a TokenTextSplitter instance
text_splitter = TokenTextSplitter(chunk_size=500, chunk_overlap=25)

pdf_loader = PyPDFLoader("/content/result.pdf") #UnstructuredPDFLoader not working

pages = pdf_loader.load_and_split()


# Split the single document into chunks using the TokenTextSplitter
# pages = text_splitter.split_text(single_document)
print(f"length of pages of single document is {len(pages)}")

# instantiate embedding model
embeddings_model = OpenAIEmbeddings()

embedder = CacheBackedEmbeddings.from_bytes_store(
    embeddings_model,
    store
)

# create vector store, we use FAISS in this case
vector_store = FAISS.from_documents(pages, embedder)

# # you could use FAISS or Chroma for vector storages
# db = Chroma.from_documents(pages, embedder)
# retriever = db.as_retriever()

# this is the entire retrieval system
medium_qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(),
    retriever=vector_store.as_retriever(), #retriever, #,
    return_source_documents=True,
    verbose=True
)


sample_prompts = ["What is the most common application of machine learning for inventory maangement?",
                  "What is the most common application of machine learning for demand forcasting in supply chain maangement?"]

for prompt in sample_prompts:

    #vanilla OpenAI Response
    response = openmixer.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens = 500)

    # RAG Augmented Response
    response_rag = medium_qa_chain({"query":prompt})

# starts from here to run extracted PDF

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Create 2 virtual GPUs with 1GB memory each
  try:
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=1024),
         tf.config.LogicalDeviceConfiguration(memory_limit=1024)])
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPU, 2 Logical GPUs


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Nov 25 20:08:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    41W / 300W |    518MiB / 16384MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# import locale
# print(locale.getpreferredencoding())

ANSI_X3.4-1968


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install transformers accelerate einops langchain xformers bitsandbytes chromadb sentence_transformers

In [ ]:
!pip install openai==0.28

In [ ]:
# !pip install light-the-torch
# !ltt install torch torchvision

In [ ]:
! pip install pypdf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer, GPTQConfig
from time import time
import chromadb
from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.document_loaders.pdf import PDFPlumberLoader, PyPDFLoader

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-creden

In [ ]:
model_id = 'meta-llama/Llama-2-7b-hf'

#device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

# separate into different batches

# per_device_train_batch_size=4,

#trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=transformers.TrainingArguments(

https://stackoverflow.com/questions/76459034/how-to-load-a-fine-tuned-peft-lora-model-based-on-llama-with-huggingface-transfo





In [ ]:
!nvidia-smi

Sat Nov 25 20:12:16 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    41W / 300W |    518MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# import torch
# torch.cuda.is_available()

In [ ]:
# ! pip install flash-attn --no-build-isolation

In [ ]:
import torch

# Check if a GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

# Your model and data operations
# model.to(device)

In [ ]:

time_1 = time()
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

quantization = GPTQConfig(bits=4,dataset = "c4", tokenizer=tokenizer)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    #quantization_config=quantization,
    device_map='auto',
    #load_in_8bit_fp32_cpu_offload=True,
    #load_in_8bit=True
)
# put the model into cuda

#model.to(device)

time_2 = time()
print(f"Prepare model, tokenizer: {round(time_2-time_1, 3)} sec.")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Prepare model, tokenizer: 62.485 sec.


In [ ]:
time_1 = time()
query_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        device_map="auto",
        batch_size=8)
time_2 = time()
print(f"Prepare pipeline: {round(time_2-time_1, 3)} sec.")

Prepare pipeline: 1.494 sec.


In [ ]:
#!pip install openai==0.28

In [ ]:
#testing OPEN AI

# sample_prompts = ["What is the most common application of machine learning for inventory maangement?",
#                   "What is the most common application of machine learning for demand forcasting in supply chain maangement?"]

# for prompt in sample_prompts:

#     #vanilla OpenAI Response
#     response = openai.Completion.create(
#         engine="text-davinci-003",
#         prompt=prompt,
#         max_tokens = 500)

#     # RAG Augmented Response
#     response_rag = medium_qa_chain({"query":prompt})

NameError: ignored

In [ ]:
def test_model(tokenizer, pipeline, prompt_to_test):
    """
    Perform a query
    print the result
    Args:
        tokenizer: the tokenizer
        pipeline: the pipeline
        prompt_to_test: the prompt
    Returns
        None
    """
    # adapted from https://huggingface.co/blog/llama2#using-transformers
    time_1 = time()
    sequences = pipeline(
        prompt_to_test,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=200,)
    time_2 = time()
    print(f"Test inference: {round(time_2-time_1, 3)} sec.")
    for seq in sequences:
        print(f"Result: {seq['generated_text']}")

In [ ]:
# skip this as it takes too much time
test_model(tokenizer,
           query_pipeline,
           "What is the most common application of machine learning for inventory management?")

Test inference: 1101.462 sec.
Result: What is the most common application of machine learning for inventory management?
Machine learning algorithms are used to analyze inventory data to predict future demand and optimize inventory levels. This can include predicting customer demand based on past sales data, forecasting seasonal fluctuations in demand, and identifying trends in customer behavior.
How can machine learning be used to improve the efficiency of inventory management?
Machine learning algorithms can be used to automate inventory management processes, such as reordering and forecasting. This can improve efficiency by reducing the time and resources required to manage inventory.
What are the benefits of using machine learning for inventory management?
The benefits of using machine learning for inventory management include improved forecasting accuracy, increased efficiency, and reduced costs. Machine learning can also help to identify opportunities for cost savings and optimize

In [ ]:
# from transformers import pipeline
# llm = pipeline(pipeline=query_pipeline)
# # checking again that everything is working fine
# llm.tokenizer.pad_token_id = llm.config.eos_token_id,
# llm(prompt="What is the most common application of machine learning for inventory management?", )

RuntimeError: ignored

In [ ]:
# Create a TokenTextSplitter instance
#text_splitter = TokenTextSplitter(chunk_size=500, chunk_overlap=25)

pdf_file_path = '/content/drive/My Drive/Colab/Transformer/result.pdf'

pdf_loader = PyPDFLoader(pdf_file_path) #UnstructuredPDFLoader not working

pages = pdf_loader.load_and_split()




# loader = TextLoader("/content/result.pdf",
#                     encoding="utf8")
# documents = loader.load()

In [ ]:
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
# all_splits = text_splitter.split_documents(documents)

In [ ]:
model_name = "bert-base-uncased" #sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}

# from transformers import BertConfig, BertModel

# # Initializing a BERT bert-base-uncased style configuration
# configuration = BertConfig()

# # Initializing a model (with random weights) from the bert-base-uncased style configuration
# model = BertModel(configuration)

#embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

.gitattributes:   0%|          | 0.00/491 [00:00<?, ?B/s]

LICENSE:   0%|          | 0.00/11.4k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

(…)kage/Data/com.apple.CoreML/model.mlmodel:   0%|          | 0.00/165k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
# db = Chroma.from_documents(pages, embeddings)
# retriever = db.as_retriever()

In [ ]:
# import torch

# # Check if a GPU is available
# if torch.cuda.is_available():
#     device = torch.device("cuda")
# else:
#     device = torch.device("cpu")

# # Your model and data operations
# # model.to(device)
#

In [ ]:
pages = pages.to(device)

AttributeError: ignored

In [ ]:
vectordb = Chroma.from_documents(documents=pages, embedding=embeddings, persist_directory="chroma_db")

NameError: ignored

In [ ]:
retriever = vectordb.as_retriever()
llm = HuggingFacePipeline(pipeline=query_pipeline)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True
)

In [ ]:
def test_rag(qa, query):
    print(f"Query: {query}\n")
    time_1 = time()
    result = qa.run(query)
    time_2 = time()
    print(f"Inference time: {round(time_2-time_1, 3)} sec.")
    print("\nResult: ", result)

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
tf.debugging.set_log_device_placement(True)
gpus = tf.config.list_logical_devices('GPU')
strategy = tf.distribute.MirroredStrategy(gpus)
with strategy.scope():
  inputs = tf.keras.layers.Input(shape=(1,))
  predictions = tf.keras.layers.Dense(1)(inputs)
  model = tf.keras.models.Model(inputs=inputs, outputs=predictions)
  model.compile(loss='mse',
                optimizer=tf.keras.optimizers.SGD(learning_rate=0.2))

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StatelessRandomGetKeyCounter in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StatelessRandomUniformV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op NoOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in d

In [ ]:
query = "What is the most common application of machine learning for inventory management? Summarize. Keep it under 200 words."
test_rag(qa, query)

Query: What is the most common application of machine learning for inventory management? Summarize. Keep it under 200 words.



> Entering new RetrievalQA chain...


OutOfMemoryError: ignored

In [ ]:
query = "WWhat is the most common application of machine learning for demand forcasting in supply chain management? Summarize. Keep it under 200 words."
test_rag(qa, query)

In [ ]:

docs = vectordb.similarity_search(query)
print(f"Query: {query}")
print(f"Retrieved documents: {len(docs)}")
for doc in docs:
    doc_details = doc.to_json()['kwargs']
    print("Source: ", doc_details['metadata']['source'])
    print("Text: ", doc_details['page_content'], "\n")